In [2]:
# !conda install -y -c conda-forge ffmpeg==7.1.1 libiconv

# !pip install torch==2.8.0 \
#     torchaudio==2.8.0

# !pip install torchcodec==0.7

# !pip install -U google-genai \
# python-dotenv \
# pyannote.audio

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [20]:
from google import genai
from google.genai import types
import wave

# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

client = genai.Client()
temperature = 2.0
top_k = 3 # 上位3個のトークンを選択
top_p = 0.6 # 上位合計60%までトークンを選択
prompt = """入力コンテキストを減らすか、別のモデルに一時的に切り替えて（Gemini 1.5 Pro から Gemini 1.5 Flash など）、動作するかどうかを確認します。しばらく待ってからリクエストを再試行してください。再試行しても問題が解決しない場合は、Google AI Studio の [フィードバックを送信] ボタンを使用してご報告ください。
"""

def genai_tts():
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-tts",
        contents=prompt,
        config=types.GenerateContentConfig(
            response_modalities=["AUDIO"],
            # system_instruction="", # モデルに対しての指示
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            speech_config=types.SpeechConfig(
                language_code="ja-JP",
                voice_config=types.VoiceConfig(
                         prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name='Kore',
                         )
                )
            ),
        )
    )
    
    data = response.candidates[0].content.parts[0].inline_data.data
    print(type(data))
    return data

data = genai_tts()
wave_file("out1.wav", data)
data = genai_tts()
wave_file("out2.wav", data)
data = genai_tts()
wave_file("out3.wav", data)

<class 'bytes'>


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 3\nPlease retry in 21.878436464s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash-tts'}, 'quotaValue': '3'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDela

In [21]:
import IPython.display
IPython.display.Audio("out1.wav")

In [22]:
IPython.display.Audio("out2.wav")

In [23]:
IPython.display.Audio("out3.wav")